In [2]:
import pandas as pd

In [ ]:
pathDonnees = "SMSSpamCollection.txt"
trainProportion = 0.5

In [3]:
df = pd.read_csv(
    pathDonnees, delimiter="\t", header=None, names=["spam", "text"]
)


In [3]:
sizeData = len(df.index)
trainSize = int(trainProportion*sizeData)
trainDF = df.sample(trainSize)
testDF = df.drop(trainDF.index)


In [4]:
trainDFSpam = trainDF[trainDF["spam"]=="spam"]
trainDFSpam["text"]=trainDFSpam["text"].str.lower()
dfSplitSpam = trainDFSpam.text.str.split(expand=True).stack().value_counts().to_frame()

trainDFHam = trainDF[trainDF["spam"]=="ham"]
trainDFHam["text"]=trainDFHam["text"].str.lower()
dfSplitHam = trainDFHam.text.str.split(expand=True).stack().value_counts().to_frame()

total = dfSplitSpam.join(dfSplitHam,lsuffix='_spam', rsuffix='_ham')
total.fillna(0.001,inplace=True)
spamTotal=total["count_spam"].sum()
hamTotal = total["count_ham"].sum()
total["vraisemblance"]=(total["count_spam"]/spamTotal)/(total["count_ham"]/hamTotal)

spamProbality = spamTotal/hamTotal

C:\Users\melos\AppData\Local\Temp\ipykernel_9184\3630862678.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trainDFSpam["text"]=trainDFSpam["text"].str.lower()
C:\Users\melos\AppData\Local\Temp\ipykernel_9184\3630862678.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trainDFHam["text"]=trainDFHam["text"].str.lower()


In [5]:
total

,count_spam,count_ham,vraisemblance
to,328,832.000,0.936732
call,168,100.000,3.991849
a,166,537.000,0.734512
your,133,223.000,1.417136
you,123,875.000,0.334012
...,...,...,...
there's,1,16.000,0.148506
prizeawaiting,1,0.001,2376.100738
rude,1,0.001,2376.100738
united,1,0.001,2376.100738


In [6]:
df = total.sort_values("vraisemblance", ascending=False).head(40)

In [7]:
from matplotlib.axis import YAxis
import plotly.express as px
import numpy as np
violin = total[(total["vraisemblance"] > 0) & (total["vraisemblance"] < 10)]
px.violin(violin, y="vraisemblance")

In [8]:
total["vraisemblance"].min()

np.float64(0.011534469601767749)

In [9]:
def TestPhraseList(phraseList,spamProba=1):
    for word in phraseList:
        vraisemblance = total["vraisemblance"].get(word,"NotFound")
        if vraisemblance!="NotFound":
            spamProba*=vraisemblance
    return(spamProba)
print(TestPhraseList(["hello","call","to","win","your","prize"]))
print(TestPhraseList(["hello","call","me","i","miss","you"]))

def TestPhrase(phrase:str,spamProba=1):
    phraseList=phrase.split()
    return TestPhraseList(phraseList,spamProba=spamProba)


print(TestPhrase("Hi how are you ? I have a wonderful proposition for you ! Gain your wonderful price by clicking on this link"))


616095.115258622
0.00023869371872073947
0.0016612990373588987


In [10]:
testDF["text"]=testDF["text"].str.split()

In [17]:
wrong = len(testDF[testDF["Finalresult"]!=testDF["spam"]].index)
FP = len(testDF[(testDF["Finalresult"]=="spam")&(testDF["spam"]=="ham")].index)
FN = len(testDF[(testDF["Finalresult"]=="ham")&(testDF["spam"]=="spam")].index)
VP = len(testDF[(testDF["Finalresult"]=="ham")&(testDF["spam"]=="ham")].index)
VN = len(testDF[(testDF["Finalresult"]=="spam")&(testDF["spam"]=="spam")].index)
wrong/len(testDF.index)
exactitude = (VN+VP)/(VN+FN+FP+VP)
print(f"Exactitude : {exactitude}")
rappel = VP /(VP+FN)
print(f"Rappel : {rappel}")
print(f"Rappel négative: {VN/(VN+FP)}")
precision = VP/(VP+FP)
print(f"Precision : {precision}")
print(f"Précision négative: {VN/(VN+FN)}")
F1Score = (2 * precision * rappel) / (precision + rappel)
print(f"F1-Score : {F1Score}")
print(f"F1-Score Négatif: {(2 * VN/(VN+FN) * VN/VN+FP )/ (VN/(VN+FN) + VN/VN+FP)}")

Exactitude : 0.9508255563531945
Rappel : 0.9837119588512645
Rappel négative: 0.7814569536423841
Precision : 0.9586466165413534
Précision négative: 0.9030612244897959
F1-Score : 0.9710175587053099
F1-Score Négatif: 0.9990392880618901
